# Akansha's dataset

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Function to calculate methylation level from a single-cell file
def process_single_cell_file(file_path):
    column_names = ["chr", "start_pos", "end_pos", "bin_id", "methylated_read_count", "total_read_count"]
    cell_data = pd.read_csv(file_path, sep="\t", header=None, names=column_names)

    # Replace dots with zeros
    cell_data = cell_data.replace(".", 0)

    # Convert methylated_read_count and total_read_count columns to numeric
    cell_data[["methylated_read_count", "total_read_count"]] = cell_data[["methylated_read_count", "total_read_count"]].apply(pd.to_numeric)

    # Calculate methylation level
    cell_data["methylation_level"] = cell_data["methylated_read_count"] / cell_data["total_read_count"]
    cell_data["methylation_level"].fillna(0, inplace=True)

    # Return methylation levels as a series
    return cell_data["methylation_level"]

# Load single-cell methylation data
data_directory = "/work/magroup/ruochiz/Data/scHiC_collection/m3c_mouse_brain/methyl/500kb"  # Replace with the path to your single-cell files directory
data_files = [os.path.join(data_directory, file) for file in os.listdir(data_directory)]

# Read methylation levels from each file into a DataFrame
methylation_data = pd.DataFrame({os.path.basename(file).split(".")[0]: process_single_cell_file(file) for file in data_files})

# Transpose the DataFrame, so each row represents a single cell
methylation_data = methylation_data.transpose()

# Scaling
scaler = StandardScaler()
scaled_data = scaler.fit_transform(methylation_data)

# PCA
pca = PCA(n_components=20)
pca_data = pca.fit_transform(scaled_data)

# KMeans clustering
n_clusters = 5  # Adjust the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
clusters = kmeans.fit_predict(pca_data)

# Plot the clusters
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=clusters, cmap="viridis")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("KMeans Clustering of Single-Cell Methylation Data (PCA)")
plt.savefig('single_cell_clusters.png',dpi=300,bbox_inches='tight')
plt.show()



FileNotFoundError: [Errno 2] No such file or directory: './data'

In [9]:
! ls

# The following block is for Bigger dataset without hiC
### Ignore for now.

In [ ]:
#import module
import tarfile
import gzip
from io import BytesIO
import pandas as pd
#declare filename
filename= "GSE131354_RAW (1).tar"
 
#open file in write mode
file_obj= tarfile.open(filename,"r")
 
# get the names of files in tar file
namelist=file_obj.getnames()
print(namelist)
#print the filenames
print("files in the tar file are:")
for name in namelist[:1]:
    print(name)
    if name.endswith('.gz'):
        print(f'Reading file: {name}')

        # Extract the .gz file from the .tar archive
        extracted_file = file_obj.extractfile(name)

        # Open the .gz file using gzip
        with gzip.open(BytesIO(extracted_file.read()), 'rt') as gz_file:
            # Read the contents of the .gz file
            df = pd.read_csv(gz_file, sep='\t',header=None)
            print(df.head())
            df.columns = ['chrom','pos','strand','seq','methylated','unmethylated','state']
            print(df['chrom'].unique())
            
            # Do something with the content, e.g., print the first 100 characters
            #print(f'Content of {name} (last 1000 characters):\n{file_content[:1000]}\n')
#close file
file_obj.close()